In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from IPython import display
import argparse
import logging
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import tsfresh
from tsfresh.feature_extraction import MinimalFCParameters
from os.path import exists

In [ ]:
## FUNCTIONS ##

#Parse String
def split_ele(x):
    a = np.array(re.findall(r"[-+]?\d*\.?\d+|[-+]?\d+", x))
    return a.astype(np.float)

def get_args():
    parser = argparse.ArgumentParser(description='Recommendations for the given song')
    parser.add_argument('-s', '--song', required=True, help='Name of Song')
    parser.add_argument('-a', '--artist', required=True, help='Name of Artist')
    return parser.parse_args()


def get_song(name_song, name_artist):
    results = sp.search(q=name_song + ' '+name_artist, type='track')
    items = results['tracks']['items']
    if len(items) > 0:
        return items[0]
    else:
        return None

def show_recommendations_for_song(song):
    results = sp.recommendations(seed_tracks=[song['id']], limit=5)
    print("Recommendations:")
    for track in results['tracks']:
        print("TRACK: ",track['name'], " - ",track['artists'][0]['name'])
        sp.add_to_queue(track['uri'])

def show_feature_based_recommendations_for_song(song):
    song_features = sp.audio_features([song['uri']])
    kwargs = {"target_danceability":song_features[0]["danceability"], "target_energy":song_features[0]['energy'], "target_key":song_features[0]['key'], "target_loudness":song_features[0]['loudness'], "target_speechiness":song_features[0]['speechiness'], "target_acousticness":song_features[0]['acousticness'], "target_instrumentalness":song_features[0]['instrumentalness'], "target_liveness":song_features[0]['liveness'], "target_valence":song_features[0]['valence'], "target_tempo":song_features[0]['tempo'], "target_time_signature":song_features[0]['time_signature']}
    results = sp.recommendations(seed_artists=None, seed_genres=None, seed_tracks=[song['id']], limit=5, country=None, **kwargs)
    print("Feature-based Recommendations:")
    for track in results['tracks']:
        print("TRACK: ",track['name'], " - ",track['artists'][0]['name'])
        sp.add_to_queue(track['uri'])


In [ ]:
# READ IN FILE ##

df = pd.read_csv("slow_dancing.csv", sep = ',', usecols=range(1,34))
pose_landmark = ['NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER', 'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT',
              'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW', 'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX', 'LEFT_THUMB',
              'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE', 'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX']
display.Image("./pose_tracking_full_body_landmarks.png")

# EXTRACT FEATURES

In [ ]:
song_name_list = ['love_girl', 'party_rock_anthem', 'lite_spots', 'girlfriends_best_friend', 'scaredy_cat', 'girl_anachronism', 'lil_darlin', 'el_invento', 'test_drive', 'weight_in_gold', 'easy', 'comethru', 'classic', 'tongue_tied', 'motley', 'knife_talk', 'feed_the_fire', 'lose_it', 'thats_what_i_like', 'humble', 'skeletons', 'stairway_to_heaven', 'slow_dancing_in_the_dark', 'peaches', 'kilby_girl', 'jupiter_love', 'woman', 'get_up', 'throwin_elbows', 'power', 'ymca']
artist_name_list = ['cnblue', 'lmfao', 'kaytranada', 'walkabout', 'dpr_ian', 'the_dresden_dolls', 'count_basie', 'jose_gonzalez_calvaire', 'ariana_grande', 'gallant', 'wheein', 'jeremy_zucker', 'mkto', 'grouplove', 'post_malone', 'drake', 'lucky_daye', 'swmrs', 'bruno_mars', 'kendrick_lamar', 'keshi', 'led_zeppelin', 'joji', 'justin_bieber', 'backseat_lovers', 'trey_songz', 'doja_cat', 'shinedown', 'excision', 'kanye', 'village_people']
num_runs = 2

In [ ]:
extracted_features = pd.DataFrame()

settings = MinimalFCParameters()

for file_num in range(1, num_runs+1):
    for index in range(len(song_name_list)):
        file_song = song_name_list[index]
        file_artist = artist_name_list[index]

        file_name = "data/"+str(file_num) + "_" + file_song + "_"+file_artist+".csv"
        
        #if file does not exist, skip it
        if(exists(file_name)):
            num_features = 10
            num_nodes = 12
            num_samples = 10

            df = pd.read_csv(file_name, sep = ',', usecols=[7, 8, 15, 16, 19, 20, 23, 24, 25, 26, 27, 28])

            #CLEAN UP DATA
            x_data = pd.DataFrame(columns=pose_landmark, index=range(len(df)))
            y_data = pd.DataFrame(columns=pose_landmark, index=range(len(df)))
            z_data = pd.DataFrame(columns=pose_landmark, index=range(len(df)))
            vis_data = pd.DataFrame(columns=pose_landmark, index=range(len(df)))

            for node in pose_landmark:
                curr = df[node]
                vals = [split_ele(x) for x in curr]
                for row in range(len(vals)):
                    x_data[node][row] = vals[row][0]
                    y_data[node][row] = vals[row][1]
                    z_data[node][row] = vals[row][2]
                    vis_data[node][row] = vals[row][3]


            #MOTION FEATURE EXTRACTION
            curr_extracted_vector = pd.DataFrame()

            #LOOP THROUGH NODES
            for node in pose_landmark:
                node_x = x_data[node]
                node_y = y_data[node]
                node_z = z_data[node]

                #Every Node (Body Part) has this set of feaures

                xname = node + "_x"
                yname = node + "_y"
                zname = node + "_z"
                
                #TODO add to this dictionary of features: approximate entropy, autocorrelation, c3, cid_ce
                #kind_to_fc_parameters = {
                #xname: {"abs_energy": None, "kurtosis": None, "mean": None, "standard_deviation": None, "maximum": None} , 
                #yname: {"abs_energy": None, "kurtosis": None, "mean": None, "standard_deviation": None, "maximum": None} ,
                #zname: {"abs_energy": None, "kurtosis": None, "mean": None, "standard_deviation": None, "maximum": None}}
                 
                
                comb = pd.DataFrame(data=[node_x, node_y, node_z], index=[xname, yname, zname]).T
                comb.rename_axis("time")
                comb["id"] = 1
                comb["time"] = comb.index

                
                #TODO instead of default_fc_parameters=settings, do custom feature set extraction using the created kind_to_fc_parameters dict
                curr_extracted = tsfresh.extract_features(comb, column_id = "id", column_sort="time", column_kind=None, column_value=None, default_fc_parameters=settings, disable_progressbar=True)
                curr_extracted_vector = pd.concat([curr_extracted_vector, curr_extracted], axis=1)
                
                #TODO (optional) append frequency info (max frequency, variance of frequency content, etc) using scipy.signal.spectrogram and argmax, std, etc. of the spectrogram

            
            #SONG FEATURE EXTRACTION

            logger = logging.getLogger()
            logging.basicConfig()
            CLIENT_ID="9793440f0a5047c59c70bcfcf91ad589"
            CLIENT_SECRET= "b66dc3a5f9f34207bebee32a25745368"
            REDIRECT_URL="http://localhost/"
            client_credentials_manager = SpotifyClientCredentials(client_id = CLIENT_ID, client_secret = CLIENT_SECRET)
            oAuth = SpotifyOAuth(client_id = CLIENT_ID, client_secret = CLIENT_SECRET, redirect_uri = REDIRECT_URL, scope = 'user-modify-playback-state')
            sp = spotipy.Spotify(auth_manager =oAuth)

            song_name = file_song
            artist_name = file_artist

            song = get_song(song_name, artist_name)
            
            #if get request didnt get anything
            if(song is None):
                continue
            
            song_features = sp.audio_features([song['uri']])
            
            audio_feature_list = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
            
            audio_feat = pd.DataFrame(columns = audio_feature_list, index = range(1, 2))

            audio_feat['danceability'] = song_features[0].get('danceability')
            audio_feat['energy'] = song_features[0].get('energy')
            audio_feat['key'] = song_features[0].get('key')
            audio_feat['loudness'] = song_features[0].get('loudness')
            audio_feat['mode'] = song_features[0].get('mode')
            audio_feat['speechiness'] = song_features[0].get('speechiness')
            audio_feat['acousticness'] = song_features[0].get('acousticness')
            audio_feat['instrumentalness'] = song_features[0].get('instrumentalness')
            audio_feat['liveness'] = song_features[0].get('liveness')
            audio_feat['valence'] = song_features[0].get('valence')
            audio_feat['tempo'] = song_features[0].get('tempo')
            audio_feat['time_signature'] = song_features[0].get('time_signature')

            curr_extracted_vector = pd.concat([curr_extracted_vector, audio_feat], axis=1)

            extracted_features = extracted_features.append(curr_extracted_vector)

In [ ]:
extracted_features

In [ ]:
extracted_features.to_csv("parser_output/extracted_features.csv")

Feature DF Format

File | Class? | Motion Features | Audio Features

...

Motion Feature Organization:

Nose x Feature 1 | Nose x Feature 2 | ... | Nose y Feature 1 | Nose y Feature 2 | .. | Nose z Feature 1 | ... | Right Foot Index  z Feature 1 | ...